In [25]:
import pandas as pd
import numpy as np

In [26]:
# 데이터 불러오기
df = pd.read_csv("mz_final.csv", encoding = 'utf-8')

# 가설3

##  M세대와 Z세대의 소비 패턴은 다를 것이다

## 3-1 구매 빈도 : 장바구니 담은 빈도와 구매 성공 빈도의 비교

In [27]:
# M세대 add_to_cart의 빈도수 
df_M_cart = df[(df["generation"] == "M")&(df["event_name"] == "add_to_cart")]
df_M_cart.count()['price'] 

195072

In [28]:
# M세대 purchase_success의 빈도수 
df_M_purchase = df[(df["generation"] == "M")&(df["event_name"] == "purchase_success")]
df_M_purchase.count()['price']

263135

In [29]:
# M세대 빈도 비교 
df_M_cart.count()['price']/df_M_purchase.count()['price']*100

74.1338096414388

In [30]:
# Z세대 add_to_cart의 빈도수 
df_Z_cart = df[(df["generation"] == "Z")&(df["event_name"] == "add_to_cart")]
df_Z_cart.count()['price'] 

30894

In [31]:
# Z세대 purchase_success의 빈도수 
df_Z_purchase = df[(df["generation"] == "Z")&(df["event_name"] == "purchase_success")]
df_Z_purchase.count()['price']

KeyboardInterrupt: 

In [ ]:
# Z세대 빈도 비교 
df_Z_cart.count()['price']/df_Z_purchase.count()['price']*100


## 3-2 세대 별 실제 구매전환율

In [ ]:
# 데이터 프레임에 조건 할당
df_M_c = df[(df["generation"] == "M")&(df["event_name"] == "add_to_cart")]
df_Z_c = df[(df["generation"] == "Z")&(df["event_name"] == "add_to_cart")]
df_M_p = df[(df["generation"] == "M")&(df["event_name"] == "purchase_success")]
df_Z_p = df[(df["generation"] == "Z")&(df["event_name"] == "purchase_success")]

In [ ]:
df_M_c = df_M_c[["user_no", "item_no", "event_name"]]
df_Z_c = df_Z_c[["user_no", "item_no", "event_name"]]
df_M_p = df_M_p[["user_no", "item_no", "event_name"]]
df_Z_p = df_Z_p[["user_no", "item_no", "event_name"]]

In [ ]:
# data merge
df_M = pd.merge(df_M_c,df_M_p, on = ["user_no", "item_no"], how='left')
df_Z = pd.merge(df_Z_c,df_Z_p, on = ["user_no", "item_no"], how='left')

### M세대 

In [ ]:
# 결측치 확인
df_M.head()

In [ ]:
# M세대 장바구니에도 담고, 구매까지 이루어진 구하기 위해 purchase_success없는 것 drop하기
df_M = df_M.dropna()
df_M.head()

In [ ]:
# M세대 장바구니에 담았다가 구매까지 일어난 횟수
df_M.shape[0]

In [ ]:
# 결과: M세대 장바구니 담긴 것 중에서 구매까지 일어난 것
df_M.shape[0]/df[(df["generation"]=="M")&(df["event_name"]=="add_to_cart")].shape[0]*100

### Z세대

In [ ]:
# 결측치 확인
df_Z.head()

In [ ]:
# Z세대 장바구니에도 담고, 구매까지 이루어진 구하기 위해 purchase_success없는 것 drop하기
df_Z = df_Z.dropna()
df_Z.head()

In [ ]:
# Z세대 장바구니에 담았다가 구매까지 일어난 횟수
df_Z.shape[0]

In [ ]:
# 결과: Z세대 장바구니 담긴 것 중에서 구매까지 일어난 것
df_Z.shape[0]/df[(df["generation"]=="Z")&(df["event_name"]=="add_to_cart")].shape[0]*100

## 3-1 세대별 장바구니 인기 카테고리 순위 차이 (카테고리1)

In [ ]:
#1. 세대 나누기
#2. add_to_cart만 남기기

df_M = df[(df["generation"] == "M")&(df["event_name"] == "add_to_cart")]
df_Z = df[(df["generation"] == "Z")&(df["event_name"] == "add_to_cart")]


#3. 세대별 카테고리1 순위 매기기 (top5)
df_M["category1_name"].value_counts() #.head(5)
df_Z["category1_name"].value_counts() #.head(5)

## 세대별 장바구니 연관분석 실시 (카테고리2)

In [ ]:
# !pip install mlxtend

In [32]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcl

from matplotlib.colors import LinearSegmentedColormap
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [33]:
# 구매 성공한 M세대만 남기기
df_M = df[(df["generation"] == "M")&(df["event_name"] == "purchase_success")]

# 필요한 컬럼만 보기
df2 = df_M[['user_no','category1_name']]

# index 설정
df3 = df2.reset_index()

In [34]:
# 유저별로 중복값이 번호 매기기(같은 유저 같은 번호)
df3 = df3.sort_values(by=["user_no"])
temp_df = df3["user_no"].drop_duplicates().to_frame("user_no")
temp_df["number"] = 1
temp_df["number"] = temp_df["number"].cumsum()
df3 = pd.merge(df3, temp_df, on="user_no")

# 컬럼 위치 변경(번호가 앞으로)
df4 = df3.reindex( columns = [ 'number','index','user_no','category1_name'  ])

In [35]:
# number가 같은 것끼리 category1_name을 묶어서, 
# 한 사람당 구매한 카테고리의 구성 찾기
df_tmp=df4[['number','category1_name']]
df_tmp_arr=[[]for i in range(86808)]
num=0
for i in df_tmp['category1_name'] :
    df_tmp_arr[df_tmp['number'][num]].append(i)
    num+=1
# 확인    
df_tmp_arr

[[],
 ['의류', '잡화', '의류', '의류'],
 ['의류', '의류'],
 ['잡화'],
 ['화장품/미용', '의류'],
 ['디지털/가전'],
 ['의류', '잡화', '잡화', '디지털/가전', '의류', '의류', '의류', '잡화', '의류', '잡화', '의류', '의류'],
 ['식품',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '잡화',
  '의류',
  '잡화',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '잡화',
  '의류',
  '잡화',
  '잡화',
  '잡화',
  '의류',
  '잡화',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '잡화',
  '의류',
  '잡화',
  '의류',
  '의류',
  '잡화',
  '의류',
  '의류',
  '잡화',
  '잡화',
  '잡화',
  '의류',
  '의류',
  '잡화',
  '의류',
  '잡화',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '잡화',
  '의류',
  '의류',
  '의류',
  '의류',
  '의류',
  '식품',
  '의류',
  '잡화',
  '의류',
  '의류',
  '의류',
  '잡화'],
 ['잡화', '의류', '의류', '의류'],
 ['의류', '의류'],
 ['의류'],
 ['생활/건강', '식품'],
 ['디지털/가전', '식품', '잡화'],
 ['의류', '의류', '의류', '의류', '의류'],
 ['잡화'],
 ['잡화', '컬처', '가구/인테리어'],
 ['잡화'],
 ['잡화', '잡화', '의류', '의류', '잡화'],
 ['의류'],
 ['의류',
  '잡화',
  '의류',
  '의류',
  '식품',
  '식품',
  '잡화',
  '

In [36]:
# number가 1부터 시작 -> 0은 null값 -> pop()으로 제거
# 한 사람당 같은 카테고리를 다수로 구매한 경우 중복 제거
df_tmp_arr.pop(0)
num=0
for i in df_tmp_arr :
    df_tmp_arr[num] = list(set(df_tmp_arr[num]))
    num+=1
#확인
df_tmp_arr

[['잡화', '의류'],
 ['의류'],
 ['잡화'],
 ['화장품/미용', '의류'],
 ['디지털/가전'],
 ['디지털/가전', '잡화', '의류'],
 ['식품', '잡화', '의류'],
 ['잡화', '의류'],
 ['의류'],
 ['의류'],
 ['생활/건강', '식품'],
 ['디지털/가전', '식품', '잡화'],
 ['의류'],
 ['잡화'],
 ['가구/인테리어', '잡화', '컬처'],
 ['잡화'],
 ['잡화', '의류'],
 ['의류'],
 ['생활/건강', '식품', '잡화', '의류'],
 ['잡화', '의류'],
 ['잡화'],
 ['컬처'],
 ['가구/인테리어'],
 ['잡화', '의류'],
 ['의류'],
 ['디지털/가전', '잡화'],
 ['잡화', '의류'],
 ['의류'],
 ['잡화', '컬처'],
 ['잡화'],
 ['잡화', '의류'],
 ['잡화'],
 ['잡화'],
 ['스포츠/레저', '의류'],
 ['잡화'],
 ['컬처', '의류'],
 ['잡화', '의류'],
 ['의류'],
 ['잡화'],
 ['의류'],
 ['의류'],
 ['잡화'],
 ['잡화'],
 ['생활/건강', '의류'],
 ['의류'],
 ['의류'],
 ['의류'],
 ['의류'],
 ['식품', '잡화', '의류'],
 ['잡화'],
 ['잡화'],
 ['디지털/가전', '의류'],
 ['의류'],
 ['화장품/미용'],
 ['디지털/가전', '식품', '가구/인테리어'],
 ['잡화', '의류'],
 ['의류'],
 ['잡화'],
 ['의류'],
 ['의류'],
 ['디지털/가전', '스포츠/레저', '잡화'],
 ['화장품/미용', '반려동물', '잡화', '생활/건강', '식품', '의류'],
 ['의류'],
 ['잡화'],
 ['잡화'],
 ['의류', '가구/인테리어'],
 ['의류'],
 ['컬처'],
 ['잡화'],
 ['잡화'],
 ['잡화', '의류'],
 ['잡화'],
 ['잡화'],
 ['잡화'],
 ['생활/

In [ ]:
# 타입 변환 코드 -> 컬럼명이 이상해짐 -> 안해도 될 거 같아요!
#df_tmp_arr=list(map(str, df_tmp_arr))

In [37]:
# 연관분석
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [38]:
te = TransactionEncoder()
te_ary = te.fit_transform(df_tmp_arr)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [42]:
# 지지도 0.01 이상인 것 출력
# *지지도(support) : 전체 항목 중(행들 중) x와 y를 모두 포함하는 경우의 비율
# (ex : 메뉴를 여러개 시킬 때, 치킨이랑 콜라가 포함되는 경우는 전체 중 얼마나 되는지)
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.066366,(가구/인테리어)
1,0.080950,(디지털/가전)
2,0.095718,(생활/건강)
3,0.025643,(스포츠/레저)
4,0.034571,(식품)
5,0.568307,(의류)
6,0.422189,(잡화)
7,0.038718,(컬처)
8,0.049927,(화장품/미용)
9,0.010760,"(디지털/가전, 가구/인테리어)"


In [41]:
# 향상도가 최소 1이상인 연관 관계 출력
# *향상도(lift) : 우연적 기회(random chance)를 벗어나기 위한 값.
# 1이라면 우연적인 경우로, x와 y의 관계가 독립이지만, 1보다 크거나 작다면 우연이 아닌 필연적인 관계에 있음.
# *향상도 = 신뢰도 / 지지도
association_rules(frequent_itemsets, metric="lift", min_threshold=1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(디지털/가전),(가구/인테리어),0.080950,0.066366,0.010760,0.132916,2.002783,0.005387,1.076752
1,(가구/인테리어),(디지털/가전),0.066366,0.080950,0.010760,0.162125,2.002783,0.005387,1.096882
2,(생활/건강),(가구/인테리어),0.095718,0.066366,0.018328,0.191479,2.885216,0.011976,1.154744
3,(가구/인테리어),(생활/건강),0.066366,0.095718,0.018328,0.276167,2.885216,0.011976,1.249297
4,(생활/건강),(디지털/가전),0.095718,0.080950,0.014584,0.152365,1.882217,0.006836,1.084252
5,(디지털/가전),(생활/건강),0.080950,0.095718,0.014584,0.180162,1.882217,0.006836,1.103001
6,(생활/건강),(화장품/미용),0.095718,0.049927,0.010057,0.105067,2.104415,0.005278,1.061613
7,(화장품/미용),(생활/건강),0.049927,0.095718,0.010057,0.201431,2.104415,0.005278,1.132377
8,"(생활/건강, 의류)",(가구/인테리어),0.041195,0.066366,0.010610,0.257550,3.880780,0.007876,1.257505
9,"(생활/건강, 가구/인테리어)",(의류),0.018328,0.568307,0.010610,0.578881,1.018607,0.000194,1.025110


In [ ]:
# 결과(지지도 0.05이상이면 연관 O, 향상도 = 1이면 우연)
## 카테고리1 -> M : 잡화를 사면 의류도 산다 -> 향상도 0.7
##              Z : 잡화를 사면 의류도 산다 -> 향상도 0.59 
## 카테고리2 -> M : 상의를 사면 신발을 산다 -> 향상도 0.7 / 상의를 사면 하의를 산다 -> 향상도 3.4
##              Z : 상의를 사면 하의를 산다 -> 향상도 1.2
## 카테고리3 -> M : 티셔츠를 사면 바지도 산다 -> 필연적 관계 
##              Z : X

## 세대별 장바구니 카테고리 (카테고리3) / 브랜드

In [40]:
# 데이터 불러오기
df = pd.read_csv("mz_final.csv", encoding = 'utf-8')

### 장바구니

In [47]:
# M세대 add_to_cart 카테고리 
df_mcart = df[(df['generation']=='M')&(df['event_name']=='add_to_cart')]
df_mcart['category3_name']
df_mcart['category3_name'].value_counts(ascending=False).head(20)

티셔츠        36926
바지         14363
셔츠/블라우스    11872
원피스        11331
샌들          6266
니트/스웨터      5999
슬리퍼         4509
청바지         3734
스커트         2966
숄더백         2946
볼캡          2871
파자마         2705
재킷          2632
양말          2587
에코백         2410
귀걸이         2202
잔/컵         2085
속옷세트        2037
목걸이         2034
크로스백        1897
Name: category3_name, dtype: int64

In [48]:
# Z세대 add_to_cart
df_zcart = df[(df['generation']=='Z')&(df['event_name']=='add_to_cart')]
df_zcart['category3_name']
df_zcart['category3_name'].value_counts(ascending=False).head(20)

티셔츠        7115
바지         2452
셔츠/블라우스    1999
니트/스웨터     1004
청바지         794
샌들          789
원피스         784
슬리퍼         697
숄더백         613
스니커즈        533
크로스백        478
양말          437
볼캡          430
재킷          425
에코백         425
목걸이         419
카디건         358
속옷세트        345
귀걸이         321
스웻셔츠        304
Name: category3_name, dtype: int64

In [55]:
# M세대 add_to_cart 브랜드
df_mcart = df[(df['generation']=='M')&(df['event_name']=='add_to_cart')]
df_mcart['brand_name']
df_mcart['brand_name'].value_counts(ascending=False).head(20)

Mardi Mercredi              3024
THOMASMORE                  2880
comfortlab                  2335
LENINA                      2145
ALVINCLO                    1921
URAGO                       1735
DUNST                       1631
FLUKE                       1536
Jo's lounge                 1465
FACADE PATTERN              1311
MONGDOL                     1257
UNLIMIT                     1140
ANOTHEROFFICE               1100
MARITHE FRANCOIS GIRBAUD    1008
Nothing written             1001
clove                        990
TOPTEN                       982
LOEIL                        978
Tagtraume                    946
락앤락                          892
Name: brand_name, dtype: int64

In [53]:
# Z세대 add_to_cart 브랜드
df_zcart = df[(df['generation']=='Z')&(df['event_name']=='add_to_cart')]
df_zcart['brand_name']
df_zcart['brand_name'].value_counts(ascending=False).head(20)

THOMASMORE                  629
Mardi Mercredi              467
comfortlab                  334
ANOTHEROFFICE               332
MARITHE FRANCOIS GIRBAUD    315
NEITHERS                    241
The Ordinary                235
LOEIL                       230
ontz                        228
ALVINCLO                    210
BLANK ROOM                  207
FLUKE                       206
URAGO                       204
TOPTEN                      199
DUNST                       193
UNLIMIT                     164
REORG                       164
Tagtraume                   162
Jo's lounge                 161
MONGDOL                     151
Name: brand_name, dtype: int64

## 세대별 가장 많이 구매한 카테고리 (카테고리3) / 브랜드

### 구매

In [58]:
# M세대 카테고리3 가장 많이 구매한 순위
# 구매 성공한 M세대만 남기기
df_m = df[(df["generation"] == "M")&(df["event_name"] == "purchase_success")]
# 카테고리3 구매 많이 구매한 순서대로
df_m['category3_name'].value_counts(ascending=False).head(20)

# M/Z세대 브랜드 가장 많이 구매한 브랜드 순위
# 위 식에 category3_name -> brand_name 으로

티셔츠        52470
바지         19174
셔츠/블라우스    13449
슬리퍼         9304
샌들          8962
원피스         8941
니트/스웨터      7154
볼캡          4943
청바지         4465
전시/행사       4113
파자마         3937
양말          3734
숄더백         3653
속옷세트        3589
에코백         3525
휴대폰 케이스     3176
브라          2763
스커트         2680
파우치         2630
잔/컵         2620
Name: category3_name, dtype: int64

In [59]:
# M세대 카테고리3 가장 많이 구매한 순위
# 구매 성공한 M세대만 남기기
df_m = df[(df["generation"] == "M")&(df["event_name"] == "purchase_success")]
# 카테고리3 구매 많이 구매한 순서대로
df_m['brand_name'].value_counts(ascending=False).head(20)

Mardi Mercredi              5356
THOMASMORE                  5236
comfortlab                  4126
그라운드시소                      3377
FLUKE                       2592
Jo's lounge                 2565
ANOTHEROFFICE               2529
ALVINCLO                    2473
MARITHE FRANCOIS GIRBAUD    2259
Nothing written             2219
clove                       2044
TAW&TOE_이누                  1862
DUNST                       1858
FACADE PATTERN              1842
UNLIMIT                     1841
URAGO                       1806
EMIS                        1546
락앤락                         1491
LENINA                      1466
The Ordinary                1443
Name: brand_name, dtype: int64

In [60]:
# Z세대 카테고리3 purchase_success 
# 구매 성공한 Z세대만 남기기
df_z = df[(df["generation"] == "Z")&(df["event_name"] == "purchase_success")]
# 카테고리3 구매 많이 구매한 순서대로
df_z['category3_name'].value_counts(ascending=False).head(20)


티셔츠        20690
바지          5232
슬리퍼         3526
셔츠/블라우스     3391
샌들          2655
니트/스웨터      1944
숄더백         1880
파우치         1803
볼캡          1769
전시/행사       1764
에코백         1593
휴대폰 케이스     1334
청바지         1256
크로스백        1226
속옷세트        1187
원피스         1172
목걸이         1018
양말           977
토트백          859
스니커즈         845
Name: category3_name, dtype: int64

In [61]:
# Z세대 카테고리3 purchase_success 
# 구매 성공한 Z세대만 남기기
df_z = df[(df["generation"] == "Z")&(df["event_name"] == "purchase_success")]
# 카테고리3 구매 많이 구매한 순서대로
df_z['brand_name'].value_counts(ascending=False).head(20)

Mardi Mercredi              3994
NEITHERS                    1482
THOMASMORE                  1404
그라운드시소                      1386
MARITHE FRANCOIS GIRBAUD    1371
Aeiou                       1254
comfortlab                  1095
The Ordinary                 834
ANOTHEROFFICE                802
kittybunnypony               705
TAW&TOE_이누                   702
typeservice                  646
ontz                         631
이굿딜                          592
EMIS                         528
Nothing written              493
MEDUSE                       488
Wave union                   483
FLUKE                        469
Tagtraume                    460
Name: brand_name, dtype: int64